<a href="https://colab.research.google.com/github/scancer-org/data-eng-preparation/blob/main/PCAM_Dataset_preparation_(HDF5_to_PyTorch).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### TODO:
# Load data - this is the major phase here
# Sample out of the data - like 100 examples for train, 20 example for test, same distribution of classes
# Have some stats on it
# Create (in PyTorch) a base model
# Train
# Eval
# Test

In [10]:
# Test labels (from original, which I'll split for train / test)
import h5py
import numpy as np
import torch
from google.colab import drive
from torch.utils import data
from os import listdir
from pathlib import Path
from torch.utils.data import DataLoader, Dataset

# anyeone interested should have this path configured already, dataset can be downloaded from GitHub / Drive (PCAM)
# Batch Size = 128
drive_base_path = 'gdrive/MyDrive/pcamv1/'
BATCH_SIZE = 128

In [7]:
drive.mount('/content/gdrive/')
!ls gdrive/MyDrive/pcamv1

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
camelyonpatch_level_2_split_test_meta.csv
camelyonpatch_level_2_split_test_x.h5
camelyonpatch_level_2_split_test_y.h5
camelyonpatch_level_2_split_train_mask.h5
camelyonpatch_level_2_split_train_meta.csv
camelyonpatch_level_2_split_train_x.h5
camelyonpatch_level_2_split_train_y.h5
camelyonpatch_level_2_split_valid_meta.csv
camelyonpatch_level_2_split_valid_x.h5
camelyonpatch_level_2_split_valid_y.h5


In [ ]:
# !gzip -d gdrive/MyDrive/pcamv1/camelyonpatch_level_2_split_valid_x.h5.gz
# !gzip -d gdrive/MyDrive/pcamv1/camelyonpatch_level_2_split_test_y.h5.gz
# !gzip -d gdrive/MyDrive/pcamv1/camelyonpatch_level_2_split_train_y.h5.gz
# !gzip -d gdrive/MyDrive/pcamv1/camelyonpatch_level_2_split_valid_y.h5.gz
# !gzip -d gdrive/MyDrive/pcamv1/camelyonpatch_level_2_split_test_x.h5.gz
# !gzip -d gdrive/MyDrive/pcamv1/camelyonpatch_level_2_split_train_mask.h5.gz
# !gzip -d gdrive/MyDrive/pcamv1/camelyonpatch_level_2_split_train_x.h5.gz

In [ ]:
# # Let's get the h5 files
# h5_file_names = [f for f in listdir(drive_base_path) if '.h5' in f]
# h5_file_names

['camelyonpatch_level_2_split_train_mask.h5',
 'camelyonpatch_level_2_split_test_x.h5',
 'camelyonpatch_level_2_split_test_y.h5',
 'camelyonpatch_level_2_split_train_y.h5',
 'camelyonpatch_level_2_split_valid_x.h5',
 'camelyonpatch_level_2_split_valid_y.h5']

In [ ]:
# # Code is taken from here https://stackoverflow.com/questions/53576113/most-efficient-way-to-use-a-large-data-set-for-pytorch

# ## TODO: should implement some kind of a loop, that will load each file (maybe each X and then each Y?), and will do the below:
# # 1. read hdf5
# # 2. make sure we can parse its values
# # 3. store them in PyTorch Datasets accordingly
# # Picked one file which already exists in my Drive (validation X)

# for h5_file_name in ['camelyonpatch_level_2_split_train_mask.h5']:
#   print("Some processing on:", h5_file_name)
#   hf = h5py.File(drive_base_path + h5_file_name, 'r')
#   # We can take its main key (one and only - X)
#   group_key = list(hf.keys())[0]
#   print(group_key)
#   # ds is the dataset now
#   ds = hf[group_key]

#   # load only one example from ds
#   x = ds[0]
#   print(x)
#   # load a subset, slice (6 examples) 
#   arr = ds[:6]
#   # print(arr)
#   hf.close()
# # In simple terms, ds can now be used as an iterator which gives images on the fly 
# # (i.e. it doesn't load anything in memory). This should make the whole run time blazing fast.

In [ ]:
# class HDF5Dataset(data.Dataset):
#     """Represents an abstract HDF5 dataset.
    
#     Input params:
#         file_path: Path to the folder containing the dataset (one or multiple HDF5 files).
#         recursive: If True, searches for h5 files in subdirectories.
#         load_data: If True, loads all the data immediately into RAM. Use this if
#             the dataset is fits into memory. Otherwise, leave this at false and 
#             the data will load lazily.
#         data_cache_size: Number of HDF5 files that can be cached in the cache (default=3).
#         transform: PyTorch transform to apply to every data instance (default=None).
#     """
#     def __init__(self, file_path, recursive, load_data, data_cache_size=3, transform=None):
#         super().__init__()
#         self.data_info = []
#         self.data_cache = {}
#         self.data_cache_size = data_cache_size
#         self.transform = transform

#         # Search for all h5 files
#         p = Path(file_path)
#         assert(p.is_dir())
#         if recursive:
#             files = sorted(p.glob('**/*.h5'))
#         else:
#             files = sorted(p.glob('*.h5'))
#         if len(files) < 1:
#             raise RuntimeError('No hdf5 datasets found')

#         for h5dataset_fp in files:
#             self._add_data_infos(str(h5dataset_fp.resolve()), load_data)
            
#     def __getitem__(self, index):
#         # get data
#         x = self.get_data("data", index)
#         if self.transform:
#             x = self.transform(x)
#         else:
#             x = torch.from_numpy(x)

#         # get label
#         y = self.get_data("label", index)
#         y = torch.from_numpy(y)
#         return (x, y)

#     def __len__(self):
#         return len(self.get_data_infos('data'))
    
#     def _add_data_infos(self, file_path, load_data):
#         with h5py.File(file_path, 'r') as h5_file:
#             # Walk through all groups, extracting datasets
#             for gname, group in h5_file.items():
#                 for ds in group[()]:
#                 #.items():
#                     # if data is not loaded its cache index is -1
#                     idx = -1
#                     if load_data:
#                         # add data to the data cache
#                         idx = self._add_to_cache(ds, file_path)
                    
#                     # type is derived from the name of the dataset; we expect the dataset
#                     # name to have a name such as 'data' or 'label' to identify its type
#                     # we also store the shape of the data in case we need it
#                     self.data_info.append({'file_path': file_path, 'type': gname, 'shape': ds.shape, 'cache_idx': idx})

#     def _load_data(self, file_path):
#         """Load data to the cache given the file
#         path and update the cache index in the
#         data_info structure.
#         """
#         with h5py.File(file_path) as h5_file:
#             for gname, group in h5_file.items():
#                 for ds in group[()]:
#                     # add data to the data cache and retrieve
#                     # the cache index
#                     idx = self._add_to_cache(ds, file_path)

#                     # find the beginning index of the hdf5 file we are looking for
#                     file_idx = next(i for i,v in enumerate(self.data_info) if v['file_path'] == file_path)

#                     # the data info should have the same index since we loaded it in the same way
#                     self.data_info[file_idx + idx]['cache_idx'] = idx

#         # remove an element from data cache if size was exceeded
#         if len(self.data_cache) > self.data_cache_size:
#             # remove one item from the cache at random
#             removal_keys = list(self.data_cache)
#             removal_keys.remove(file_path)
#             self.data_cache.pop(removal_keys[0])
#             # remove invalid cache_idx
#             self.data_info = [{'file_path': di['file_path'], 'type': di['type'], 'shape': di['shape'], 'cache_idx': -1} if di['file_path'] == removal_keys[0] else di for di in self.data_info]

#     def _add_to_cache(self, data, file_path):
#         """Adds data to the cache and returns its index. There is one cache
#         list for every file_path, containing all datasets in that file.
#         """
#         if file_path not in self.data_cache:
#             self.data_cache[file_path] = [data]
#         else:
#             self.data_cache[file_path].append(data)
#         return len(self.data_cache[file_path]) - 1

#     def get_data_infos(self, type):
#         """Get data infos belonging to a certain type of data.
#         """
#         data_info_type = [di for di in self.data_info if di['type'] == type]
#         return data_info_type

#     def get_data(self, type, i):
#         """Call this function anytime you want to access a chunk of data from the
#             dataset. This will make sure that the data is loaded in case it is
#             not part of the data cache.
#         """
#         fp = self.get_data_infos(type)[i]['file_path']
#         print(fp)
#         if fp not in self.data_cache:
#             self._load_data(fp)
        
#         # get new cache_idx assigned by _load_data_info
#         cache_idx = self.get_data_infos(type)[i]['cache_idx']
#         return self.data_cache[fp][cache_idx]

In [8]:
class H5Dataset_From_2_Files_X_Y(Dataset):
    def __init__(self, path):
        self.file_path = path
        self.dataset_x = None
        self.dataset_y = None
        ### Going to read the X part of the dataset - it's a different file
        with h5py.File(self.file_path + '_x.h5', 'r') as filex:
            self.dataset_x_len = len(filex['x'])

        ### Going to read the y part of the dataset - it's a different file
        with h5py.File(self.file_path + '_y.h5', 'r') as filey:
            self.dataset_y_len = len(filey['y'])

    def __getitem__(self, index):
        if self.dataset_x is None:
            self.dataset_x = h5py.File(self.file_path +'_x.h5', 'r')['x']
        if self.dataset_y is None:
            self.dataset_y = h5py.File(self.file_path +'_y.h5', 'r')['y']
        return (self.dataset_x[index], self.dataset_y[index])

    def __len__(self):
        assert self.dataset_x_len == self.dataset_y_len
        return self.dataset_x_len

In [13]:
dataloader_params = {'batch_size': 128, 'shuffle': True, 'num_workers': 2}

train_path = drive_base_path + 'camelyonpatch_level_2_split_train'
val_path = drive_base_path + 'camelyonpatch_level_2_split_valid'
test_path = drive_base_path + 'camelyonpatch_level_2_split_test'

test_dataset = H5Dataset_From_2_Files_X_Y(test_path)
test_dataloader = DataLoader(test_dataset, **dataloader_params)

val_dataset = H5Dataset_From_2_Files_X_Y(val_path)
val_dataloader = DataLoader(val_dataset, **dataloader_params)

train_dataset = H5Dataset_From_2_Files_X_Y(train_path)
train_dataloader = DataLoader(train_dataset, **dataloader_params)

In [15]:
for x, y in test_dataloader:
  print(x.shape)
  print(y.shape)  
  break

for x, y in val_dataloader:
  print(x.shape)
  print(y.shape)
  break

for x, y in train_dataloader:
  print(x.shape)
  print(y.shape)
  break

torch.Size([128, 96, 96, 3])
torch.Size([128, 1, 1, 1])
torch.Size([128, 96, 96, 3])
torch.Size([128, 1, 1, 1])
torch.Size([128, 96, 96, 3])
torch.Size([128, 1, 1, 1])


In [ ]:
# for x, y in test_dataloader:
#   print(x[0][:][:][:].shape)
#   break

torch.Size([96, 96, 3])
